## Fine-tune opt-6.7b with QLoRA

### Checkout my [Twitter(@rohanpaul_ai)](https://twitter.com/rohanpaul_ai) for daily LLM bits

In [ ]:
# !pip install -q bitsandbytes datasets accelerate loralib transformers peft

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
model_id = "facebook/opt-6.7b"

try:
    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                 return_dict=True,
                                                 load_in_8bit=True,
                                                 device_map='auto')
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Clear any cached memory to optimize GPU usage
    torch.cuda.empty_cache()

except Exception as e:
    print(f"Error loading model: {e}")

## Preparing the model

- Freeze all our layers

- cast the layer-norm in `float32` for stability. 

- Also cast the output of the last layer in `float32` for the same reasons.

In [ ]:
def prepare_model_for_finetuning(model):
    for param in model.parameters():
        param.requires_grad = False
        if param.ndim == 1:
            param.data = param.data.to(torch.float32)

    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()

    class CastOutputToFloat(nn.Sequential):
        def forward(self, x): return super().forward(x).to(torch.float32)

    model.lm_head = CastOutputToFloat(model.lm_head)

# In the main script, assuming 'model' is the model instance
prepare_model_for_finetuning(model)


## LoRA

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
    f"trainable params: {trainable_params} || "
    f"all params: {all_param} || "
    f"trainable%: {100 * trainable_params / all_param}"
)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

### Training

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")

data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("username/opt-6.7b-lora", use_auth_token=True)

## Load adapters from the Hub

In [ ]:
peft_model_id = "username/opt-6.7b-lora"

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

## Inference

In [ ]:
def generate_text(input_text, model, tokenizer, max_new_tokens=50):
    batch = tokenizer(input_text, return_tensors='pt')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=max_new_tokens)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Example usage
input_text = "The AI is coming for the  "
output_text = generate_text(input_text, model, tokenizer)
print(output_text)
